In [ ]:


import pandas as pd
import numpy as np
import os
import glob
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns

# Directory path
directory_path = '/mnt/c/2024-Data/SCH_asthma_114'  # Replace with your directory path
output_directory = '/mnt/c/2024-Data/Cluster_Outputs'  # Directory to save outputs
os.makedirs(output_directory, exist_ok=True)

# Find all Excel files in the directory
excel_files = glob.glob(os.path.join(directory_path, '*.xlsx'))

# List to store data from all files
all_data = []

# Loop over each Excel file and append data to the list
for file_path in excel_files:
    print(f"Processing file: {file_path}")
    
    # Read the Excel file with error handling
    try:
        df = pd.read_excel(file_path, engine='openpyxl', skiprows=2)
    except Exception as e:
        print(f"Error reading {file_path}: {e}. Skipping this file.")
        continue  # Skip to the next file if there's an error

    # column names
    possible_columns = ['A01', 'A04', 'C01', 'C02']
    
    # Find matching columns
    selected_columns = [col for col in possible_columns if col in df.columns]
    
    # Ensure all selected columns are found, otherwise skip this file
    if len(selected_columns) < len(possible_columns):
        print(f"Warning: Not all required columns found in {file_path}. Skipping this file.")
        continue
    
    # Select the relevant columns and drop rows with NaN values
    filtered_df = df[selected_columns].dropna()
    
    # Check if filtered_df is empty after dropping NaN values
    if filtered_df.empty:
        print(f"Warning: No valid data after dropping NaN values in {file_path}. Skipping this file.")
        continue
    
    # Rename columns for easier reference
    filtered_df.columns = ['ID', 'Date', 'Morning PEFR', 'Afternoon PEFR']
    
    # Convert 'Date' to datetime format and handle NaT values
    filtered_df['Date'] = pd.to_datetime(filtered_df['Date'], format='%Y-%m-%d', errors='coerce')
    filtered_df = filtered_df.dropna(subset=['Date'])  

    # Check if filtered_df is empty after dropping invalid dates
    if filtered_df.empty:
        print(f"Warning: No valid dates found in {file_path} after conversion. Skipping this file.")
        continue

    # Create a 'Days' column based on the difference from the minimum date in each file
    filtered_df['Days'] = (filtered_df['Date'] - filtered_df['Date'].min()).dt.days
    
    # Append this DataFrame to the list
    all_data.append(filtered_df[['Days', 'Morning PEFR', 'Afternoon PEFR']])

# Combine all data into a single DataFrame
combined_df = pd.concat(all_data, ignore_index=True)

# Check if combined data is empty
if combined_df.empty:
    print("No data available for clustering. Exiting.")
else:
    # Standardize the combined data
    scaler = StandardScaler()
    scaled_data = scaler.fit_transform(combined_df)

    # Optional - Use the Elbow Method 
    inertia = []
    K = range(1, 11)
    for k in K:
        kmeans = KMeans(n_clusters=k, random_state=42)
        kmeans.fit(scaled_data)
        inertia.append(kmeans.inertia_)

    # Plot and Save the Elbow Method
    plt.figure(figsize=(8, 6))
    plt.plot(K, inertia, 'bx-')
    plt.xlabel('Number of clusters')
    plt.ylabel('Inertia')
    plt.title('Elbow Method For Optimal k (Combined Data)')
    elbow_plot_path = os.path.join(output_directory, "elbow_plot.png")
    plt.savefig(elbow_plot_path)
    plt.show()

    # Apply K-Means Clustering with
    num_clusters = 3  # Set based on Elbow plot or manually
    kmeans = KMeans(n_clusters=num_clusters, random_state=42)
    combined_df['Cluster'] = kmeans.fit_predict(scaled_data)

    # Save the combined clustered data to a new Excel file
    output_path = os.path.join(output_directory, "combined_clustered_data.xlsx")
    combined_df.to_excel(output_path, index=False)
    print(f"Clustered combined data saved to: {output_path}")

    # 3D Visualization of Clusters
    fig = plt.figure(figsize=(10, 8))
    ax = fig.add_subplot(111, projection='3d')
    scatter = ax.scatter(combined_df['Days'], 
                         combined_df['Morning PEFR'], 
                         combined_df['Afternoon PEFR'], 
                         c=combined_df['Cluster'], cmap='viridis', marker='o')
    ax.set_xlabel('Days')
    ax.set_ylabel('Morning PEFR')
    ax.set_zlabel('Afternoon PEFR')
    plt.colorbar(scatter, label='Cluster')
    plt.title('3D Visualization of Clusters Based on Days, Morning PEFR, and Afternoon PEFR')
    cluster_plot_path = os.path.join(output_directory, "3d_cluster_plot.png")
    plt.savefig(cluster_plot_path)
    plt.show()

    # Trend Analysis Over Time
    trend_df = combined_df.groupby(['Days', 'Cluster']).mean().reset_index()

    plt.figure(figsize=(12, 6))
    sns.lineplot(data=trend_df, x='Days', y='Morning PEFR', hue='Cluster', palette='viridis')
    plt.title('Trend Analysis of Morning PEFR Over Time by Cluster')
    plt.xlabel('Days')
    plt.ylabel('Morning PEFR')
    trend_morning_path = os.path.join(output_directory, "trend_morning_pefr.png")
    plt.savefig(trend_morning_path)
    plt.show()

    plt.figure(figsize=(12, 6))
    sns.lineplot(data=trend_df, x='Days', y='Afternoon PEFR', hue='Cluster', palette='viridis')
    plt.title('Trend Analysis of Afternoon PEFR Over Time by Cluster')
    plt.xlabel('Days')
    plt.ylabel('Afternoon PEFR')
    trend_afternoon_path = os.path.join(output_directory, "trend_afternoon_pefr.png")
    plt.savefig(trend_afternoon_path)
    plt.show()

    # Display and Save Cluster Centers
    cluster_centers = kmeans.cluster_centers_
    cluster_centers_original = scaler.inverse_transform(cluster_centers)
    centers_df = pd.DataFrame(cluster_centers_original, columns=['Days', 'Morning PEFR', 'Afternoon PEFR'])
    centers_df['Cluster'] = range(len(centers_df))

    centers_path = os.path.join(output_directory, "cluster_centers.xlsx")
    centers_df.to_excel(centers_path, index=False)
    print(f"Cluster centers saved to: {centers_path}")

    # Visualize the cluster centers in the 3D plot
    fig = plt.figure(figsize=(10, 8))
    ax = fig.add_subplot(111, projection='3d')
    scatter = ax.scatter(combined_df['Days'], 
                         combined_df['Morning PEFR'], 
                         combined_df['Afternoon PEFR'], 
                         c=combined_df['Cluster'], cmap='viridis', marker='o', alpha=0.5)
    ax.scatter(centers_df['Days'], 
               centers_df['Morning PEFR'], 
               centers_df['Afternoon PEFR'], 
               c='red', s=300, marker='X', label='Cluster Centers')
    ax.set_xlabel('Days')
    ax.set_ylabel('Morning PEFR')
    ax.set_zlabel('Afternoon PEFR')
    plt.colorbar(scatter, label='Cluster')
    plt.title('3D Visualization of Clusters and Cluster Centers')
    cluster_centers_plot_path = os.path.join(output_directory, "3d_cluster_centers_plot.png")
    plt.savefig(cluster_centers_plot_path)
    plt.legend()
    plt.show()

   


    
   
